Initializations:

In [1]:
%reload_ext snakeviz
%matplotlib inline
%run SBRG2.py

#Core
##Pauli Algebra
Pauli operator class. A Pauli operator is denoted as
$$\sigma=\mathrm{i}^{x\cdot z}\prod_{i\in Xs}X_i\prod_{i\in Zs}Z_i=\mathrm{i}^{x\cdot z}\prod_{i}X_i^{x_i}Z_i^{z_i},$$
where $x_i=\delta(i\in Xs)$, $z_i=\delta(i\in Zs)$ are vectors of 0,1, and $X_i$, $Z_i$ are the Pauli operator on site $i$.

Pauli operators can be created
- from Xs, Zs lists: `Pauli(Xs, Zs)`,
- or from dict of indices: `Pauli({i:mu, ...})`,
- or from list of indices: `Pauli([m0, mu1, ...])`.

In [2]:
[Pauli([0,1],[1,2]), Pauli({0:1,1:2,2:3}), Pauli([1,2,3])]

[<Xs:[0, 1] Zs:[1, 2]>, <Xs:[0, 1] Zs:[1, 2]>, <Xs:[0, 1] Zs:[1, 2]>]

The commutativity of two Pauli operators can be checked by `is_commute`

In [3]:
is_commute(Pauli([0,1,0]),Pauli([1,2,3]))

False

Merge two Pauli operators by indices. Coefficients are not calculated here. The coefficient can be restored from the commutation relations later.

In [4]:
pdot(Pauli([0,1,0]),Pauli([1,2,3]))

<Xs:[0] Zs:[1, 2]>

### Index Processing Unit (IPU)
Pauli indices are prossed in Fortran object IPU. Set two index arrays A and B by `ipu.setab(A, B)`.

In [2]:
A = Pauli([3,2,1,2,1])
print(A)
ipu.setab(array(A.Xs), array(A.Zs))

<Xs:[1, 2, 3, 4] Zs:[0, 1, 3]>


In [3]:
print(ipu.a[:ipu.na])
print(ipu.b[:ipu.nb])

[1 2 3 4]
[0 1 3]


Count the number of overlap indices by `dups = ipu.overlap()`.

In [4]:
ipu.overlap()

2

List operations
- Merge indices by the method `ipu.merge()`, 
- Combine indices by the method `ipu.combine()`,
- Intersect indices by the method `ipu.intersect()`.

The result is stored in `ipu.c[:ipu.nc]`.

In [3]:
ipu.merge()
print(ipu.c[:ipu.nc])
ipu.combine()
print(ipu.c[:ipu.nc])
ipu.intersect()
print(ipu.c[:ipu.nc])

[0 2 4]
[0 1 2 3 4]
[1 3]


###Pauli Monomial
A term of Pauli operator `Term.mat` with a constant multiple `Term.val`.

In [5]:
[Term(Pauli([1,2])),Term(Pauli([1,2]),-3.2)]

[1.0 <Xs:[0, 1] Zs:[1]>, -3.2 <Xs:[0, 1] Zs:[1]>]

Pauli monomial can be negated simpliy:

In [6]:
-Term(Pauli([1,2]))

-1.0 <Xs:[0, 1] Zs:[1]>

Two Pauli monomials are compared by and only by their Pauli operators:

In [7]:
print(Term(Pauli([1,2]),2.) == Term(Pauli([1,2]),1.))
print(Term(Pauli([1,1]),2.) < Term(Pauli([1,2]),1.))

True
True


Consider $\sigma_A=\mathrm{i}^{x_A\cdot z_A} X_A Z_A$, $\sigma_B=\mathrm{i}^{x_B\cdot z_B} X_B Z_B$, then the product reads:
$$\sigma_A\sigma_B=\mathrm{i}^{x_A\cdot z_A+x_B\cdot z_B+2 z_A\cdot x_B}\quad (X_A X_B) (Z_A Z_B)=\mathrm{i}^{x_A\cdot z_A+x_B\cdot z_B-x_C\cdot z_C+2 z_A\cdot x_B}\quad \mathrm{i}^{x_C\cdot z_C}X_C Z_C=\mathrm{i}^{n^{C}_{AB}}\; \sigma_C,$$
where $X_C=X_AX_B$, $Z_C=Z_AZ_B$ and $z_C = (z_A+z_B)\mod 2$. The power of $\mathrm{i}$ is
$$n^{C}_{AB}=x_A\cdot z_A+x_B\cdot z_B-x_C\cdot z_C+2 z_A\cdot x_B$$

In [11]:
dot(Term(Pauli([1,2])),Term(Pauli([3,3])))

1.0 <Xs:[0, 1] Zs:[0]>

In [9]:
idot(Term(Pauli([2,0])),Term(Pauli([3,3])))

-1.0 <Xs:[0] Zs:[1]>

###Find Clifford Rotation
`findR(A, phybits)` finds the Clifford rotations to diagonalize the Pauli operator `A` in the physical qubits specified by the set `phybits`.

In [8]:
A = Pauli([0,0,3,0,0])
print(A)
findR(A,{1,2,3})

<Xs:[] Zs:[2]>


[]

##Pauli Polynomial
Pauli polynomial `Poly` is a sum of Pauli monomial terms, with additional data keeping the order of Pauli operators and values. It can be created from a list of terms.

In [9]:
H = Poly([Term(Pauli([1,3,0]),1.0),
      Term(Pauli([0,1,1]),0.5),
      Term(Pauli([3,0,0]),0.2),
      Term(Pauli([0,3,3]),0.8)])

In [10]:
H.terms

[0.2 <Xs:[] Zs:[0]>,
 0.8 <Xs:[] Zs:[1, 2]>,
 1.0 <Xs:[0] Zs:[1]>,
 0.5 <Xs:[1, 2] Zs:[]>]

In [11]:
H.descending()

[1.0 <Xs:[0] Zs:[1]>,
 0.8 <Xs:[] Zs:[1, 2]>,
 0.5 <Xs:[1, 2] Zs:[]>,
 0.2 <Xs:[] Zs:[0]>]

In [12]:
H.imap

{0: {0.2 <Xs:[] Zs:[0]>, 1.0 <Xs:[0] Zs:[1]>},
 1: {0.8 <Xs:[] Zs:[1, 2]>, 1.0 <Xs:[0] Zs:[1]>, 0.5 <Xs:[1, 2] Zs:[]>},
 2: {0.8 <Xs:[] Zs:[1, 2]>, 0.5 <Xs:[1, 2] Zs:[]>}}

In [21]:
Rs = Term(Pauli([0],[0,1]),-1.)
print(Rs)

-1.0 <Xs:[0] Zs:[0, 1]>


In [14]:
H.forward(Rs)

In [15]:
H + [Term(Pauli([],[0]),-0.1)]

((), (0,)) ((), (0,))


In [16]:
H.terms

[0.9 <Xs:[] Zs:[0]>,
 0.8 <Xs:[] Zs:[1, 2]>,
 -0.2 <Xs:[0] Zs:[1]>,
 -0.5 <Xs:[0, 1, 2] Zs:[0, 1]>]

In [17]:
H.imap

{0: {0.9 <Xs:[] Zs:[0]>, -0.2 <Xs:[0] Zs:[1]>, -0.5 <Xs:[0, 1, 2] Zs:[0, 1]>},
 1: {0.8 <Xs:[] Zs:[1, 2]>,
  -0.2 <Xs:[0] Zs:[1]>,
  -0.5 <Xs:[0, 1, 2] Zs:[0, 1]>},
 2: {0.8 <Xs:[] Zs:[1, 2]>, -0.5 <Xs:[0, 1, 2] Zs:[0, 1]>}}

In [18]:
H.backward(Rs)

In [19]:
H.terms

[0.2 <Xs:[] Zs:[0]>,
 0.8 <Xs:[] Zs:[1, 2]>,
 0.9 <Xs:[0] Zs:[1]>,
 0.5 <Xs:[1, 2] Zs:[]>]

In [20]:
H.imap

{0: {0.2 <Xs:[] Zs:[0]>, 0.9 <Xs:[0] Zs:[1]>},
 1: {0.8 <Xs:[] Zs:[1, 2]>, 0.9 <Xs:[0] Zs:[1]>, 0.5 <Xs:[1, 2] Zs:[]>},
 2: {0.8 <Xs:[] Zs:[1, 2]>, 0.5 <Xs:[1, 2] Zs:[]>}}

## SBRG

In [16]:
class SBRG:
    def __init__(self, model):
        self.tol = 1.e-8
        self.max_rate = 2.
        self.L = model.L
        self.H = Poly(model.terms)
        self.EHM = []
        self.phybits = list(range(self.L))
        self.bit = 0
        self.trash = []
    def _findR(self, mat):
        if len(mat.Xs) > 0: # if X or Y exists, use it to pivot the rotation
            self.bit = mat.Xs[0] # take first off-diag qubit
            return [idot(Term(Pauli([],[self.bit])), Term(mat))]
        else: # if only Z
            if len(mat.Zs) > 1:
                for self.bit in mat.Zs: # find first Z in phybits
                    if (self.bit in self.phybits):
                        tmp = Term(Pauli([self.bit],[])) # set intermediate term
                        return [idot(tmp, Term(mat)), idot(Term(Pauli([],[self.bit])), tmp)]
            elif len(mat.Zs) == 1:
                self.bit = mat.Zs[0]
        return []
    def _perturbation(self, H0, offdiag):
        pass
    def _nextstep(self):
        if len(self.phybits) == 0: # return if no physical bits
            return self
        # get leading energy scale
        H0 = self.H.UVscale
        if abs(H0.val) == 0.: # if leading scale vanishes
            self.phybits = [] # quench physical space
            return self
        # find Clifford rotations
        Rs = self._findR(H0.mat)
        self.EHM.extend(Rs) # add to EHM
        self.H.forward(Rs) # apply to H
        offdiag = [term for term in self.H.imap[self.bit] if self.bit in term.mat.Xs]
        perturbation(H0, offdiag)

In [17]:
random.seed(2)
system = SBRG(TFIsing(8,J=1.,K=1.,h=1.,alpha=1.))

In [18]:
system._nextstep()

[0.3312296038076123 <Xs:[5, 6, 7] Zs:[5, 6]>, 0.8941609955139828 <Xs:[4, 5] Zs:[6]>]


In [15]:
system.H.UVscale

0.998046196887083 <Xs:[] Zs:[5]>

In [8]:
system.EHM

[1.0 <Xs:[5] Zs:[5, 6]>, -1.0 <Xs:[5] Zs:[5]>]

In [11]:
system.H.imap

{0: {0.36958493230522654 <Xs:[] Zs:[0]>,
  0.5380200687341565 <Xs:[] Zs:[0, 1]>,
  0.020838411395880302 <Xs:[] Zs:[0, 7]>,
  0.45625696845938785 <Xs:[0, 1] Zs:[]>,
  0.9818507992081988 <Xs:[0, 7] Zs:[]>},
 1: {0.5380200687341565 <Xs:[] Zs:[0, 1]>,
  0.2274901235770828 <Xs:[] Zs:[1]>,
  0.5007067613106101 <Xs:[] Zs:[1, 2]>,
  0.45625696845938785 <Xs:[0, 1] Zs:[]>,
  0.25402820954890987 <Xs:[1, 2] Zs:[]>},
 2: {0.5007067613106101 <Xs:[] Zs:[1, 2]>,
  0.07863267363247047 <Xs:[] Zs:[2]>,
  0.9842381317519938 <Xs:[] Zs:[2, 3]>,
  0.25402820954890987 <Xs:[1, 2] Zs:[]>,
  0.47459746510478773 <Xs:[2, 3] Zs:[]>},
 3: {0.9842381317519938 <Xs:[] Zs:[2, 3]>,
  0.4009793653291901 <Xs:[] Zs:[3]>,
  0.480551921518253 <Xs:[] Zs:[3, 4]>,
  0.47459746510478773 <Xs:[2, 3] Zs:[]>,
  0.3810229403594042 <Xs:[3, 4] Zs:[]>},
 4: {0.480551921518253 <Xs:[] Zs:[3, 4]>,
  0.27870080758089066 <Xs:[] Zs:[4]>,
  0.5261969973075329 <Xs:[] Zs:[4, 5, 6]>,
  0.3810229403594042 <Xs:[3, 4] Zs:[]>,
  0.8941609955139828 <Xs